Quick Drawing is a kaggle challenge for classifying the sketch pictures. The total number of category is 340 and 
there are 50M pictures in the dataset. Each catogory has 25,000 pictures. The difficulty is the drawing might be 
incomplete or the drawing does not match the label. We are trying to develop a CNN to solve the issue.   

According to the challenge, there are 4 kinds of dataset. 

    - Raw data (.ndjson)
    - Simplified drawings (.ndjson)
    - Binary data (.bin)
    - Bitmap data (.npy)
Raw data and Simplified drawings are provided in a vector information of strokes. Bitmap data is rendered into 
28 * 28 grayscale bitmap in the numpy format.  

Maybe we could consider the raw data to improve the performance since it inlcudes the sequence information.


In [1]:
import torch
import time
import torch
import os
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
from torch.utils import data
import QDdata as QD
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
import resnet as RN
from torchvision import transforms

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 300

print('using device:', device)


using device: cuda


# Data Augmentation

In [3]:
trans = transforms.Compose([transforms.RandomHorizontalFlip(0.5), transforms.ToTensor()])

# Baseline 28X28

In [7]:
train_dataset28 = QD.QDDetection(count = 40 * 3000*9 ,images_category = 3000*9,transforms = trans)
loader_dataset28 = QD.QDDetection(count = 40 * 512,start = 3000*9, images_category = 3000)
train_loader28 = DataLoader(dataset=train_dataset28, batch_size=128, num_workers=2, shuffle=True)
loader_val28 = DataLoader(dataset=loader_dataset28, batch_size=128,shuffle=True)

def train28(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        t1 = time.time()
        for t, (x, y) in enumerate(train_loader28):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            t2 = time.time()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f time:%.4f' % (t, loss.item(),t2 - t1))
                check_accuracy(loader_val28, model)
                t1 = time.time()


learning_rate28 = 1e-3

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
model28 = nn.Sequential(
    nn.Conv2d(1,64,3,padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64,64,3,padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64,128,3,padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,256,3,padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Flatten(),
    nn.Linear(12544, 40),
)

# you can use Nesterov momentum in optim.SGD
optimizer28 = optim.Adam(model28.parameters(), lr=learning_rate28)

QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"base28"}
train28(model28, optimizer28, epochs=5)

  5%|▌         | 2/40 [00:00<00:02, 14.73it/s]

Total class number:40


  8%|▊         | 3/40 [00:00<00:01, 19.81it/s]

Total number of items: 1080000 1080000
Total class number:40


100%|██████████| 40/40 [00:01<00:00, 24.61it/s]

Total number of items: 20480 20480


Iteration 0, loss = 3.7049 time:0.1277
Got correct (2.50,4.64)
Iteration 300, loss = 2.0869 time:9.1140
Got correct (41.77,52.32)
Iteration 600, loss = 1.9954 time:9.0852
Got correct (48.26,58.28)
Iteration 900, loss = 1.8500 time:9.0782
Got correct (49.98,60.35)
Iteration 1200, loss = 1.7586 time:9.0969
Got correct (52.89,62.77)
Iteration 1500, loss = 1.6334 time:9.0801
Got correct (53.84,63.72)
Iteration 1800, loss = 1.5295 time:9.0829
Got correct (54.31,64.10)
Iteration 2100, loss = 1.8036 time:9.0794
Got correct (56.26,66.05)
Iteration 2400, loss = 1.5614 time:9.1045
Got correct (54.32,64.24)
Iteration 2700, loss = 1.5502 time:9.0863
Got correct (55.03,64.93)
Iteration 3000, loss = 1.3300 time:9.0854
Got correct (58.02,67.37)
Iteration 3300, loss = 1.5667 time:9.0991
Got correct (57.50,67.26)
Iteration 3600, loss = 1.3900 time:9.0788
Got correct (57.70,67.32)
Iteration 3900, loss = 1.5099 time:9.0781
Got correct (58.89,68.35)
Iteration 4200, loss = 1.3182 time:9.0794
Got correct (5

Iteration 1500, loss = 1.2185 time:9.0483
Got correct (64.57,73.66)
Iteration 1800, loss = 1.0545 time:9.0507
Got correct (65.13,74.02)
Iteration 2100, loss = 1.2256 time:9.0556
Got correct (63.79,72.82)
Iteration 2400, loss = 1.0680 time:9.0597
Got correct (64.66,73.58)
Iteration 2700, loss = 1.2256 time:9.0521
Got correct (64.69,73.77)
Iteration 3000, loss = 1.1089 time:9.0851
Got correct (64.99,73.76)
Iteration 3300, loss = 1.0431 time:9.0528
Got correct (58.90,68.74)
Iteration 3600, loss = 1.2255 time:9.0726
Got correct (65.01,73.88)
Iteration 3900, loss = 1.2228 time:9.0500
Got correct (65.00,73.98)
Iteration 4200, loss = 1.0233 time:9.0623
Got correct (64.86,73.86)
Iteration 4500, loss = 1.3860 time:9.0501
Got correct (65.15,74.04)
Iteration 4800, loss = 1.0105 time:9.0699
Got correct (64.24,73.23)
Iteration 5100, loss = 1.1112 time:9.0673
Got correct (58.38,68.32)
Iteration 5400, loss = 1.2659 time:9.0652
Got correct (64.66,73.63)
Iteration 5700, loss = 1.1431 time:9.0583
Got co

# Baseline model (96X96)

In [8]:
base_learning_rate = 1e-4

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
base_model = nn.Sequential(
    nn.Conv2d(1,64,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.Conv2d(64,128,3,stride = 2,padding=1),
    nn.ReLU(),
    nn.Conv2d(128,128,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 1),
    nn.Conv2d(128,256,3,stride = 2,padding=1),
    nn.ReLU(),
    nn.Conv2d(256,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 1),
    Flatten(),
    nn.Linear(9216,4096),
    nn.ReLU(),
    nn.Linear(4096,40)
)


base_optimizer = optim.Adam(base_model.parameters(),lr=base_learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"base96"}
train(base_model, base_optimizer, epochs=5)

data10.npy label10.npy
Total number of items: 120000
data0.npy label0.npy
Total number of items: 120000
Epoch 0,Iteration 0, loss = 3.7450 time 0.0205
Got correct (3.53,5.17)
Epoch 0,Iteration 300, loss = 1.2792 time 28.0616
Got correct (61.99,70.94)
Epoch 0,Iteration 600, loss = 1.0423 time 28.0804
Got correct (69.12,77.29)
Epoch 0,Iteration 900, loss = 1.0038 time 28.0575
Got correct (71.35,79.32)
data1.npy label1.npy
Total number of items: 120000
Epoch 0,Iteration 0, loss = 0.8431 time 0.0176
Got correct (72.17,79.73)
Epoch 0,Iteration 300, loss = 0.8628 time 28.0100
Got correct (74.15,81.44)
Epoch 0,Iteration 600, loss = 0.8026 time 28.0134
Got correct (75.27,82.31)
Epoch 0,Iteration 900, loss = 0.7856 time 28.0212
Got correct (75.58,82.67)
data2.npy label2.npy
Total number of items: 120000
Epoch 0,Iteration 0, loss = 1.1083 time 0.0173
Got correct (75.21,82.32)
Epoch 0,Iteration 300, loss = 0.7144 time 28.0192
Got correct (75.46,82.66)
Epoch 0,Iteration 600, loss = 0.9568 time 27.

# Sketch-A-Net model(256 X 256)
* The following code is based on Sketch-A-Net that Beats Humans.

In [4]:
train_dataset_sn = QD.QDStrokeDataset(count = 40 * 3000 ,images_category = 3000*9,transforms = trans)
loader_dataset_sn = QD.QDStrokeDataset(count = 40 * 512,start = 3000*9, images_category = 3000)
train_loader_sn = DataLoader(dataset=train_dataset_sn, batch_size=128, num_workers=2, shuffle=True)
loader_val_sn = DataLoader(dataset=loader_dataset_sn, batch_size=128,shuffle=True)

SN_learning_rate = 1e-3

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
SN_model = nn.Sequential(
    nn.Conv2d(1,64,15,stride = 3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),
    
    nn.Conv2d(35,128,5,stride = 1,padding=0),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),
    
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),    
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),

    Flatten(),
    nn.Linear(256*7*7,512),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(512,512),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(512,40)
)


SN_optimizer = optim.Adam(SN_model.parameters(),weight_decay = 1e-5,lr=SN_learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"sketchanet"}
train(SN_model, SN_optimizer, epochs=5)

  0%|          | 0/40 [00:00<?, ?it/s]

Total class number:40


AttributeError: 'str' object has no attribute 'shape'

# Run test samples

In [ ]:
def evaluate_test(test_data, model,y):
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        meanAP = []
        meanAP3 = []
        for i ,x in enumerate(test_data):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #y = y_label.to(device=device, dtype=torch.long)
            y_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        return (m1,m3)

model = base_model.to(device=device)
checkpoint = torch.load("model_parameter99.3972598.87986")
model.load_state_dict(checkpoint)

class_name = QD.QDloadtest(name = "./data1.npy")
class_name = class_name.class_names

class_accuracy = []

for i,name in enumerate(tqdm(class_name)):
    test_data = QD.QDloadtest(name=os.path.join("../cs230/pic96",name+".npz"),image_size=(96,96))
    test_loader = DataLoader(dataset=test_data, batch_size=128, num_workers=2, shuffle=False)
    y = np.ones(len(test_data))*i
    m1,m3 = evaluate_test(test_loader,model,y)
    print(name,m1,m3)
    class_accuracy.append((name,str(m1),str(m3)))
    
with open('class_accuracy.csv',"w+") as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(["class", "meanAP1", "meanAP3"])  # write header
    writer.writerows(class_accuracy)


1) 96*96, resnet50, epoch 10, GSD. 75.2%
2) 96*96, resnet50, epoch 10, GSD  79.9% model_parametertensor(91.6667)
3) 96*96, resnet50, epoch 10, Adam model_resnet_92.317_90


* convert simplified test dataset to pixel drawsing

Check the accuracy based on the class.

Create test_simplified.
load compressed(96*96) 1.968482494354248
load non compressed(96*96)  0.31299567222595215
load non compressed(28*28) 0.04715704917907715


In [ ]:
for i in range(10,11):
    data = QD.QDcreateData()
    data.create(start=i*3000,dir_name="../cs230/pic96")
    del data